### Salish Sea Oil Spill Scenario: Turn Point

This notebook describes oil particle tracking for a spill scenario at Turn Point in the winter (Jan 2018). The code if for translating particle tracking from *ocean parcels* using the *Salish Sea Cast* grid into input forcing files for *Atlantis* for the *Atlantis* box grid.

Surface wind forcing has been temporarily removed.

In [1]:
import sys
import os
import math
from parcels import Field, FieldSet, AdvectionRK4, ParticleSet, JITParticle, ParticleFile, plotTrajectoriesFile, Variable, ScipyParticle
import numpy as np
from datetime import timedelta
sys.path.append('/ocean/rlovindeer/Atlantis/ssam_oceanparcels/Parcels_Utils/particle_tracking/parcels/')
from util.file_plotting import bec_plotTrajectoriesFile
from util.seed_particles import get_particles, get_release_times
from util.parse_wildcards import parse_wildcards

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2926/libparcels_random_5b47af32-755e-48ae-a759-a52c813803d5.so


In [2]:
file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen"}
print(scenario[file_id])

 1


5b_Turn_Point_Diluted_bitumen


In [3]:
def DecayParticle(particle, fieldset, time):
    dt = particle.dt
    print(dt)
    field_decay_value = fieldset.decay
    print(field_decay_value)
    decay = math.exp(-1.0 * dt/field_decay_value)
    particle.decay_value = particle.decay_value * decay

In [4]:
data_path = '/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/'
grid_path = '/ocean/rlovindeer/Atlantis/Physics/Grids/'

u_data_path = data_path + '2018-01*URaw_variables.nc'
v_data_path = data_path + '2018-01*VRaw_variables.nc'
u_paths = parse_wildcards(u_data_path, 'u')
v_paths = parse_wildcards(v_data_path, 'v')

In [5]:
# With Salish Sea NEMO Model Grid, Geo-location and Bathymetry, v17-02
filenames = {'U': {'lon': grid_path + 'ubcSSnBathymetryV17-02_a29d_efc9_4047.nc',
                   'lat': grid_path + 'ubcSSnBathymetryV17-02_a29d_efc9_4047.nc',
                   'data': u_paths},
             'V': {'lon': grid_path + 'ubcSSnBathymetryV17-02_a29d_efc9_4047.nc',
                   'lat': grid_path + 'ubcSSnBathymetryV17-02_a29d_efc9_4047.nc',
                   'data': v_paths}}

In [6]:
variables = {'U': 'uVelocity',
             'V': 'vVelocity'}

In [7]:
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
fieldset = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
fieldset.add_constant('decay', 1.0 * 3600.0)

In [8]:
# with wind
#fieldset_sum = FieldSet(U=fieldset.U+(wind_fieldset.U), V=fieldset.V+(wind_fieldset.V))
# without wind
fieldset_sum = FieldSet(U=fieldset.U, V=fieldset.V)

In [9]:
class MyParticle(ScipyParticle):
    initial_time = -100
    decay_value = Variable('decay_value', dtype=np.float32, initial=1.0)

In [10]:
# Particle Features
num_particles_per_day = 100
feature_release_index = 0
input_shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/Shape_Scenarios/" + scenario[file_id] + ".shp"
release_depth = -0.1

### Date and season of spill release
Winter: Dec - Mar
Summer: Jul - Aug  
Check Susan about when to do simulation

In [10]:
release_start_time = '2018-01-01'  
release_end_time = '2018-01-02'

In [11]:
release_start_time = np.datetime64(release_start_time)
release_end_time = np.datetime64(release_end_time)
time_origin = fieldset.U.grid.time_origin.time_origin

In [12]:
[release_times, p, num_particles] = get_release_times(time_origin, num_particles_per_day, release_start_time, release_end_time)

-1800.0
100
100


In [13]:
pset = get_particles(fieldset, num_particles, input_shapefile_name, MyParticle, feature_release_index, release_times, release_depth)

Using shapefile /ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/Shape_Scenarios/5b_Turn_Point_Diluted_bitumen.shp
Number of sites :  1
Up to 0 of 1
80
100
Sucessfully seeded particles

num_attempts = 2
[-123.29342181743614, -123.29308906571126, -123.29331391769186, -123.29342967807801, -123.29367213484532, -123.29322765621787, -123.29364427002147, -123.29273589844243, -123.29375256140635, -123.29293599436784, -123.2934616546045, -123.29338335532181, -123.29266825116741, -123.2943451858446, -123.2928542047529, -123.2929631309421, -123.29277942014751, -123.29292112731557, -123.29359648571949, -123.29295838609143, -123.29428289559226, -123.29323960240134, -123.29263010660989, -123.2934757478005, -123.29369012056402, -123.2939903332198, -123.29297097706512, -123.29360714377957, -123.29338257654626, -123.29448186484314, -123.29328417344985, -123.29329525299856, -123.29328557645024, -123.29263194828697, -123.29315580384579, -123.29380042864285, -123.2936453805364, -123.29312418205214,

In [14]:
decay_kernel = pset.Kernel(DecayParticle)
my_kernel = AdvectionRK4 + decay_kernel

In [15]:
output_file_name = 'Oil_disperse' + scenario[file_id] + '.nc'
print(output_file_name)

Oil_disperse5b_Turn_Point_Diluted_bitumen.nc


In [16]:
try:
    os.system('rm ' + output_file_name)
except:
    pass

In [17]:
## Output properties
output_file = pset.ParticleFile(name= output_file_name, outputdt = timedelta(minutes = 60))
pset.execute(my_kernel,                 # the kernel (which defines how particles move)
             runtime=timedelta(hours = 24*7),    # the total length of the run
             dt = timedelta(minutes = 60),      # the timestep of the kernel
             output_file = output_file)  # the file name and the time step of the outputs
output_file.close()

3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0

INFO: Temporary output files are stored in out-JMZROUDC.
INFO: You can use "parcels_convert_npydir_to_netcdf out-JMZROUDC" to convert these to a NetCDF file during the run.
N/A% (0 of 604800.0) |                   | Elapsed Time: 0:00:00 ETA:  --:--:--

1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0

 15% (93600.0 of 604800.0) |##           | Elapsed Time: 0:00:00 ETA:   0:00:03

1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0

...

1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0

 99% (601200.0 of 604800.0) |########### | Elapsed Time: 0:01:36 ETA:   0:00:00

1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0

100% (604800.0 of 604800.0) |############| Elapsed Time: 0:01:37 Time:  0:01:37


1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
1800.0
3600.0
